## Code source :

In [7]:
from tkinter import *
import tkinter as tk
import random as rd #pour le choix aléatoire des mots
from tkinter.messagebox import * 
import pickle #sera utilisé pour enregistrer le score et l'historique
import datetime as dt #sera utilisé pour la date dans l'historique

lettres = []
for i in range(26): 
    lettres.append(chr(ord('A')+i)) #on ajoute toutes les lettres de l'alphabet à la liste lettres
    
class MonBouton(Button):   #classe pour les boutons 
    def __init__(self, parent1,parent2,texte, *args, **kwargs):
        Button.__init__(self, parent2,text=texte, *args, **kwargs)
        self.__texte=texte
        self.__parent=parent1
    def statut(self): #cette fonction sera appelée lors du clic sur le clavier virtuel
        if len(self.__texte)>=2: #s'il ne s'agit pas d'un bouton du clavier virtuel
            pass #on ne fait rien
        else: #si c'est un bouton du clavier virtuel
            mot=self.__parent.get_mot() #on récupère le mot en cours
            self.__parent.set_lettre(self.__texte) #on communique l'info de la lettre cliquée à la classe FenetrePrincipale
            if self.__texte not in mot: #si la lettre cliquée n'est pas dans le mot :
                self.config(state=DISABLED) #on désactive le bouton 
            self.__parent.suite_du_jeu() #on appelle ensuite la méthode suite_du_jeu de FenetrePrincipale ...
                                         #... qui va utiliser les infos communiquées pour continuer le jeu
    
class ZoneAffichage(Label): #Affichage du pendu, hérite de la classe Label
    #Cette nouvelle classe n'est pas vraiment nécessaire, on peut utiliser directement Label 
    # mais cela permet d'avoir un nom explicite 
    def __init__(self, *args, **kwargs):
        Label.__init__(self, *args, **kwargs)
    
        
class FenetrePrincipale(Tk): #classe FenetrePrincipale
    def __init__(self):
        Tk.__init__(self)
        self.title('Jeu du pendu')
        #Initialisation de l'interface
        self.__liste_mots=[] #liste des labels pour l'affichage des mots
        self.__affichage=[] #liste des labels pour l'affichage du pendu
        self.__boutons_clavier=[] #liste des boutons du clavier
        self.__mots=[] #liste des mots importés du fichier texte
        self.__mot_cherche='' #le mot cherché
        # self. mot_en_cours est le mot cherché mais auquel on élimine au fur et à mesure les lettres ...
        #... qui sont trouvées pour que les tests d'appartenance d'une lettre ne se fassent pas ...
        #... sur les lettres déjà trouvées mais bien sur les lettres qui restent à trouver
        self.__mot_en_cours=''  
        self.__mot_affiché='' # le mot qui est affiché à l'écran
        self.__lettre='' # la lettre cliquée par le joueur, cette info est donnée par la classe MonBouton
        self.__essais_restants=8 #le nombre d'essais restants
        self.__frame_commande=Frame(self) #le Frame qui contiendra les boutons Nouvelle partie, Quitter, etc 
        self.__frame_commande.pack(side=TOP)
        self.__nom=StringVar() #pseudo du joueur tapé dans la zone "pseudo :"
        
        
        self.__score={} #dictionnaire des scores des différents joueurs
        self.__historique=[] #liste des différentes parties jouées
        
        #Bouton "Nouvelle partie"
        self.__bouton_new_partie=MonBouton(self,self.__frame_commande,'Nouvelle partie',command=self.entree_joueur)
        self.__bouton_new_partie.pack(side=LEFT,padx=3,pady=3)
        #Bouton "Quitter"
        self.__bouton_quitter=MonBouton(self,self.__frame_commande,'Quitter',command=self.destroy)
        self.__bouton_quitter.pack(side=LEFT,padx=3,pady=3)
        #Bouton "Fin de session"
        self.__fin_session=MonBouton(self,self.__frame_commande,'Fin de session',command=self.fin_session)
        self.__fin_session.pack(side=LEFT,padx=3,pady=3)
        self.__fin_session.config(state=DISABLED) #on désactive le bouton au début
        self.__frame_affichage=Frame(self) #Frame pour l'affichage du pendu
        self.__frame_affichage.pack(side=TOP) 
        
        self.__nouvelle_partie=True #expliqué dans le compte rendu, partie modélisation et mise en oeuvre
        
        self.afficher('debut.png')
        
        self.__frame_mot=Frame(self) #Frame pour l'affichage du mot
        self.__frame_nom=Frame(self) #Frame pour l'entrée du nom joueur
        
        self.__frame_entree=Frame(self) #Frame contenant le clavier virtuel
        self.__entree_texte = Entry(self.__frame_nom, textvariable= self.__nom) #permet la saisie du nom du joueur
        self.__label_joueur=Label(self.__frame_nom) #Label pour inviter le joueur à écrire son nom
        self.__label_joueur.config(text='Entrez votre pseudo :')
        self.__Bouton_OK=MonBouton(self,self.__frame_nom,'OK',command=self.verificat) #Bouton OK après la saisie du nom
        self.__frame_commande_joueur=Frame(self) #Frame pour les commandes joueur (Score, Historique, etc)
        self.__label_nom_joueur=Label(self.__frame_commande_joueur) #Label qui contiendra le nom du joueur en cours
        self.__Bouton_Score=MonBouton(self,self.__frame_commande_joueur,'Score',command=self.afficher_score) #Score
        self.__Bouton_Réinitialiser=MonBouton(self,self.__frame_commande_joueur,'Réinitialiser',command=self.reinitialiser)
        self.__Bouton_Historique=MonBouton(self,self.__frame_commande_joueur,'Historique',command=self.afficher_historique)
        
        self.chargerMots() #on charge les mots
        
    def set_mot_en_cours(self,new_mot): #permet de changer le mot en cours
        self.__mot_en_cours=new_mot
        
    def set_lettre(self,l): #permet de changer la lettre en cours
        self.__lettre=l
        
        
    def verification(self): #permet de retourner un booléen selon si le mot est trouvé ou pas
        if self.__mot_cherche==self.__mot_affiché: # si le mot cherché est égal au mot affiché
            return True 
        else : return False
        
    def suite_du_jeu(self): #méthode appelée par la classe MonBouton à chaque clic sur une lettre du clavier
        if self.__essais_restants>=1: #s'il y a plus d'1 essais restants
            if self.__lettre in self.__mot_en_cours: #si la lettre cliquée figure dans le mot
                #on remplace l'astérisque par la lettre :
                n=self.__mot_en_cours.index(self.__lettre) 
                l=list(self.__mot_affiché)
                l[n]=self.__lettre
                self.__mot_affiché=''.join(l)
                l2=list(self.__mot_en_cours)
                l2[n]='*'
                self.set_mot_en_cours(''.join(l2))
                if self.verification(): #si le mot est trouvé, càd que le joueur a trouvé la dernière lettre manquante
                    suite_texte='\nCliquer sur Nouvelle partie pour jouer à nouveau'
                    #on affiche un message Bravo ... etc
                    self.zoneMot('BRAVO, vous avez trouvé le mot: '+self.__mot_cherche+suite_texte) 
                    nom=self.__nom.get() #on récupère le pseudo du joueur
                    (a,b)=self.__score[nom] #on récupère le nb de parties gagnées et le nb total de parties ...
                    a,b=a+1,b+1 # ... et on y ajoute 1
                    self.__score[nom]=(a,b) #on met à jour le score
                    self.enregistrer_score() #on enregistre le score dans le fichier 'score'
                    #on crée une ligne historique avec le mot clé 'Succès' et la date ... :
                    x=dt.datetime.now() 
                    z=x.strftime('%d/%m/%Y %H:%M')
                    texte_histo=nom+':'+'Mot : '+self.__mot_cherche+', '+'Succès '+z 
                    self.__historique.append(texte_histo) # ... et on l'ajoute dans l'historique
                    self.enregistrer_historique() #on enregistre l'historique dans le fichier 'historique'
                    self.desactiver_boutons_clavier() #on désactive tous les boutons clavier
                if not self.verification(): #si le mot n'est pas encore trouvé
                    self.zoneMot(self.__mot_affiché) #on affiche juste la lettre à la place de l'astérique
            else : #si la lettre cliquée n'est pas dans le mot
                self.__essais_restants=self.__essais_restants-1 #on soustrait 1 aux essais restants
                self.afficher('pendu{}.png'.format(8-self.__essais_restants)) #on ajoute un élément au pendu
                
        if self.__essais_restants==0: #s'il ne reste plus d'essais restants, càd, le joueur a perdu
            self.afficher('pendu8.png') #on affiche le pendu dans sa totalité
            self.fin_partie() #on appelle la méthode fin_partie 
            
    def fin_partie(self): #cette méthode est appelée lorsque le joueur a perdu
        suite_texte='\nCliquer sur Nouvelle partie pour jouer à nouveau'
        #on affiche un message 'Vous avez perdu ...' :
        self.zoneMot('Vous avez perdu, le mot était: '+self.__mot_cherche+suite_texte) 
        nom=self.__nom.get() #on récupère le nom du joueur
        (a,b)=self.__score[nom] #on récupère le nb de parties gagnées et le nb total de parties ...
        b=b+1 #... et on ajoute 1 au nb total de parties sans toucher au nb de parties gagnées
        self.__score[nom]=(a,b) #on met à jour le score
        self.enregistrer_score() #on enregistre le score
        #on crée une ligne historique avec le mot clé 'Echec' et la date ... :
        x=dt.datetime.now()
        z=x.strftime('%d/%m/%Y %H:%M')
        texte_histo=nom+':'+'Mot : '+self.__mot_cherche+', '+'Échec '+z 
        self.__historique.append(texte_histo) # ... et on l'ajoute à l'historique
        self.enregistrer_historique() #on enregistre l'historique
        self.desactiver_boutons_clavier() #on désactive tous les boutons du clavier
        
    def afficher_clavier(self): #permet d'afficher le clavier 
        self.__frame_entree.pack(side=TOP)
        n=26
        for i in range(26):
            b=MonBouton(self,self.__frame_entree,lettres[i]) #on crée un MonBouton dans le frame pour le clavier
            b.config(command=b.statut)
            # on le met à la bonne place :
            if i<=20:
                b.grid(row=(i//7),column=i%7,columnspan=1,padx=2,pady=2) 
            if i>=21:
                b.grid(row=(i//7),column=1+i%7,columnspan=1,padx=2,pady=2)
            self.__boutons_clavier.append(b) #on l'ajoute à la liste des boutons clavier   
            
    def desactiver_boutons_clavier(self): #permet de désactiver tous les boutons clavier
        for i in self.__boutons_clavier: 
            i.config(state=DISABLED)
            
    def desafficher_commande_joueur(self): #permet de masquer le menu joueur
        self.__frame_commande_joueur.pack_forget()
        self.__label_nom_joueur.pack_forget()
        self.__Bouton_Score.pack_forget()
        self.__Bouton_Historique.pack_forget()
        self.__Bouton_Réinitialiser.pack_forget()
            
    def afficher_commande_joueur(self): 
        #permet d'afficher le menu joueur lorsque le joueur a rentré son pseudo et il appuie sur OK
        #le menu contient le nom du joueur et les boutons Score, Historique et Réinitialiser
        self.__frame_commande_joueur.pack(side=TOP)
        texte='Joueur : '+self.__nom.get()
        self.__label_nom_joueur.config(text=texte)
        self.__label_nom_joueur.pack(side=LEFT,padx=3,pady=3)
        self.__Bouton_Score.pack(side=LEFT,padx=3,pady=3)
        self.__Bouton_Historique.pack(side=LEFT,padx=3,pady=3)
        self.__Bouton_Réinitialiser.pack(side=LEFT,padx=3,pady=3)  
    
    def clean_labels(self): #permet de nettoyer la zone où l'on affiche le mot pour éviter une superposition
        for i in self.__liste_mots: i.destroy()
            
    def clean_ecran(self): #permet d'effacer complètement le pendu 
        for i in self.__affichage: i.destroy()
        self.afficher('vide.png')
        
    def fin_session(self): #méthode appelée lorsqu'on appuie sur le bouton Fin de session
        self.clean_labels() # on nettoie la zone d'affichage du mot
        #on masque tout ce qui se situe en-dessous de la zone d'affichage du pendu :
        self.__Bouton_OK.pack_forget() 
        self.__entree_texte.pack_forget()
        self.__entree_texte.pack_forget()
        self.__frame_nom.pack_forget()
        self.__frame_entree.pack_forget()
        self.__frame_mot.pack_forget()
        #on affiche une image qui contient un texte d'accueil :
        self.afficher('debut.png')
        self.__nouvelle_partie=True #on remet le booléen à True
        self.__nom.set('') #on vide la variable contenant le nom joueur
        self.desafficher_commande_joueur() #on masque aussi le menu joueur
        self.__fin_session.config(state=DISABLED) #on désactive le bouton
        
    def enregistrement_joueur(self): 
        #méthode appelée à l'intérieur de la méthode nouvelle_partie
        nom=self.__nom.get() #on récupère le nom du joueur
        self.importer_score() #on importe les scores du fichier 'score'
        if nom not in list(self.__score.keys()): #si le nom du joueur ne figure pas dans le dictionnaire des scores ...
            self.__score[nom]=(0,0) #... on crée une nouvelle clé correspondant à ce joueur dans le dictionnaire des scores
        
    def importer_historique(self): #permet d'importer l'historique depuis le fichier 'historique'
        with open('historique', 'rb') as fichier:
            liste_histo=pickle.Unpickler(fichier)
            self.__historique=liste_histo.load()
        
    def enregistrer_historique(self): #permet d'enregistrer l'historique dans le fichier 'historique'
        with open('historique', 'wb') as fichier:
            liste_hist=pickle.Pickler(fichier)
            liste_hist.dump(self.__historique)
            
    def afficher_historique(self): 
        #permet d'afficher l'historique lorsque le joueur appuie sur le bouton Historique
        nom=self.__nom.get() #on récupère le nom du joueur
        n=len(nom)
        #on crée une chaîne de caractères qui contient tout l'historique ...
        hist_affiche='' 
        for i in self.__historique:
            if i[:n+1]==nom+':':
                hist_affiche+=i[n+1:]+'\n'
        #... et ensuite on affiche une nouvelle fenêtre qui affiche la chaîne de caractères: 
        fenetre_hist = Toplevel() 
        fenetre_hist.title('Votre historique')
        scrollbar = Scrollbar(fenetre_hist) #widget qui permet d'avoir la barre de défilement
        scrollbar.pack(side = RIGHT, fill = Y) 
        T=Text(fenetre_hist)
        scrollbar.config(command=T.yview)
        T.config(yscrollcommand=scrollbar.set)
        T.pack()
        T.insert(END,hist_affiche)
        T.config(state='disabled')
        fenetre_hist.focus_set()                        

        
    def importer_score(self): #permet d'importer le score depuis le fichier 'score'
        with open('score','rb') as fich:
            dict_score=pickle.Unpickler(fich)
            self.__score=dict_score.load()
            
    def enregistrer_score(self): #permet d'enregistrer le score dans le fichier 'score'
        with open('score','wb') as fiche:
            dict_score=pickle.Pickler(fiche)
            dict_score.dump(self.__score)
            
    def afficher_score(self): 
        #permet d'afficher le score lorsque le joueur appuie sur le bouton Score
        nom=self.__nom.get() #on récupère le nom du joueur ...
        #... et on récupère les valeurs correspondantes dans le dictionnaire des scores :
        a=self.__score[nom][0] #nb de parties gagnées
        b=self.__score[nom][1] #nb total de parties
        if b==0: #si le joueur n'a jamais joué de parties ou s'il a réinitialisé ces données:
            showinfo('Votre score','Votre score est de 0 %') #on affiche un score de 0%
        #sinon on affiche le score calculé en pourcentage :
        else:showinfo('Votre score','Votre score est de {} %'.format(int(a*100/b))) 
        
    def entree_joueur(self): #méthode appelée par le bouton Nouvelle Partie
        if self.__nouvelle_partie: #si c'est une 'première' nouvelle partie
            #on affiche la zone permettant de rentrer le nom du joueur :
            self.__frame_nom.pack(side=TOP)
            self.__label_joueur.pack(side=LEFT)
            self.__entree_texte.pack(side=LEFT)
            self.__Bouton_OK.pack(side=LEFT)
            self.__nouvelle_partie=False
        else: #si c'est juste une nouvelle partie d'un joueur déjà enregistré :
            self.nouvelle_partie() #on appelle la méthode nouvelle_partie
            
    def reinitialiser(self): 
        #permet de réinitialiser les données d'un joueur (Score et Historique) lorsqu'il appuie sur le bouton Réinitialiser
        #on lui demande la confirmation via une boîte de dialogue :
        bool_reinit=askyesnocancel(title="Confirmation",message='Réinitialiser toutes vos données ?')
        if bool_reinit==True: #s'il répond oui à la confirmation
            #on supprime tous les éléments correspondants à son nom dans la liste self.__historique:
            nom=self.__nom.get()
            n=len(nom)
            s=[]
            for i in self.__historique:
                if (nom+':') in i:
                    s.append(i)
            for el in s:
                self.__historique.remove(el)
        self.__score[nom]=(0,0) #on met à 0 le score
        self.enregistrer_score() #on enregistre le score
        self.enregistrer_historique() #on enregistre l'historique
        
    def verificat(self):
        #méthode appelée lorsque le joueur appuie sur OK après avoir rentré son pseudo
        #si le joueur a laissé le champ vide, on affiche un message d'erreur :
        if self.__nom.get()=='': 
            showwarning('Erreur','Le champ ne peut pas être vide.')
        #s'il a utilisé le caractère ':' qui est utilisé comme séparateur interne, on affiche un message d'erreur :
        elif ':' in self.__nom.get():
            showwarning('Erreur','Les deux points sont interdits pour le pseudo.')
        else: #si le pseudo est rentré correctement, on masque le champ et le bouton OK
            self.__Bouton_OK.pack_forget()
            self.__entree_texte.pack_forget()
            self.__entree_texte.pack_forget()
            self.__frame_nom.pack_forget()
            self.nouvelle_partie() #on appelle la méthode nouvelle_partie
            self.__fin_session.config(state=NORMAL) #on active le bouton 'Fin de session'
        
    def zoneMot(self,mot):
        #permet d'afficher un mot dans la zone dédiée
        self.clean_labels() #on nettoie par précaution pour éviter des superpositions ...
        # ... et ensuite on affiche le label "mot":
        lmot = Label(master=self.__frame_mot) #on crée un label
        lmot.pack() #on le packe
        lmot.config(text=mot) #on le configure pour afficher le mot
        self.__liste_mots.append(lmot) #on l'ajoute à la liste des labels "mot"
        
    def chargerMots(self):#permet de charger les mots depuis le fichier txt des mots
        f = open('mots.txt','r')
        s = f.read()
        self.__mots = s.split('\n')
        f.close()
        
    def nouveauMot(self): #permet de générer aléatoirement un mot
        mot=rd.choice(self.__mots)
        self.__mot_cherche=mot
        self.__mot_en_cours=mot
        self.__mot_affiché='*'*len(mot)
        
    def get_mot(self): #permet de récupérer le mot en cours
        return self.__mot_en_cours
        
    def afficher(self,nom_image): #permet d'afficher une image dans la zone d'affichage du pendu
        photo = PhotoImage(file=nom_image,master=self)
        label = ZoneAffichage(master=self.__frame_affichage,image=photo)
        label.image = photo 
        label.grid(row=0)
        self.__affichage.append(label)
        
    def nouvelle_partie(self): #permet de faire une nouvelle partie une fois qu'un joueur a rentré son nom
        #cette méthode est appelée par la méthode verificat
        self.desafficher_commande_joueur() #on masque le menu joueur
        self.__frame_mot.pack(side=TOP) #on packe le frame mot
        self.__essais_restants=8 #on remet le nb d'essais restants à 8
        self.clean_labels() #on supprime les anciens labels "mot"
        self.clean_ecran() #on efface le pendu
        self.nouveauMot() #on choisit un nouveau mot aléatoirement
        self.zoneMot(self.__mot_affiché) #on affiche les astérisques correspondant au mot choisi
        self.afficher_clavier() #on affiche le clavier
        self.afficher_commande_joueur() #on affiche le menu joueur
        self.importer_score() #on importe le score depuis le fichier 'score'
        self.importer_historique() #on importe l'historique depuis le fichier 'historique'
        self.enregistrement_joueur() #on enregistre le joueur (si c'est un nouveau joueur)
        

f=FenetrePrincipale()
f.focus_set() 
f.mainloop()

#essai

TclError: no display name and no $DISPLAY environment variable